# Options Data

---

In [1]:
# importing libraries
from datetime import datetime, timezone, timedelta, time
import pandas as pd
import numpy as np
import csv
import time as tm
import re
import json
from tqdm import tqdm
import logging
import timeit
import concurrent.futures


In [2]:
year = 2024
month = ''

In [3]:
# df = pd.read_csv(f'data/{year}/NiftySpotData-{month}{year}-1min-Signal-Expiry.csv')
df = pd.read_csv(f'D:/all_data/data/{year}/NiftySpotData-{year}-1min-Signal-Expiry.csv')

df["Datetime"] = pd.to_datetime(df["Datetime"], format='%Y-%m-%d %H:%M:%S%z')
df['closest_expiry'] = pd.to_datetime(df['closest_expiry'], format='%Y-%m-%d %H:%M:%S%z')

# # Convert 'Datetime' to datetime and add timezone information '+05:30'
# df['Datetime'] = pd.to_datetime(df['Datetime'], format='%Y-%m-%d %H:%M:%S')
# df['Datetime'] = df['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S') + '+05:30'

# # Convert 'closest_expiry' to datetime and add time component '00:00:00+00:00'
# df['closest_expiry'] = pd.to_datetime(df['closest_expiry'], format='%Y-%m-%d')
# df['closest_expiry'] = df['closest_expiry'].dt.strftime('%Y-%m-%d') + ' 00:00:00+00:00'

# # Convert 'Datetime' back to datetime with timezone information
# df['Datetime'] = pd.to_datetime(df['Datetime'], format='%Y-%m-%d %H:%M:%S%z')

# # Convert 'closest_expiry' back to datetime with timezone information
# df['closest_expiry'] = pd.to_datetime(df['closest_expiry'], format='%Y-%m-%d %H:%M:%S%z')


try:
    # df drop PL PH, PL_val, PH_val, slope_ph, slope_pl, upper, lower, upos, dnos, IsPivotHigh
    df.drop(['PL', 'PH', 'PL_val', 'PH_val', 'slope_ph', 'slope_pl', 'upper', 'lower', 'upos', 'dnos', 'IsPivotHigh', 'Slope'], axis=1, inplace=True)
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
except:
    pass

# Initialize all columns with NaN
df = df.assign(
    position='', atmSP=np.nan, wingCall=np.nan, wingPut=np.nan,sotmcall=np.nan,sotmput=np.nan,botmcall=np.nan,botmput=np.nan , legPriceOrignal1=np.nan, legPriceOrignal2=np.nan,
    legPriceOrignal3=np.nan, legPriceOrignal4=np.nan, legPriceOrignal5=np.nan, legPriceOrignal6=np.nan, legPriceOrignal7=np.nan, legPriceOrignal8=np.nan, lpos1=np.nan, lpos2=np.nan, lpos3=np.nan, lpos4=np.nan,lpos5=np.nan, lpos6=np.nan, lpos7=np.nan, lpos8=np.nan,
    legAfterPos1=np.nan, legAfterPos2=np.nan, legAfterPos3=np.nan, legAfterPos4=np.nan,legAfterPos5=np.nan, legAfterPos6=np.nan, legAfterPos7=np.nan, legAfterPos8=np.nan, legAfterPosDiff1=np.nan,
    legAfterPosDiff2=np.nan, legAfterPosDiff3=np.nan, legAfterPosDiff4=np.nan,legAfterPosDiff5=np.nan,
    legAfterPosDiff6=np.nan, legAfterPosDiff7=np.nan, legAfterPosDiff8=np.nan, legPriceFinal1=np.nan,
    legPriceFinal2=np.nan, legPriceFinal3=np.nan, legPriceFinal4=np.nan,legPriceFinal5=np.nan,
    legPriceFinal6=np.nan, legPriceFinal7=np.nan, legPriceFinal8=np.nan, m2m1=np.nan, m2m2=np.nan, m2m3=np.nan,
    m2m4=np.nan,m2m5=np.nan, m2m6=np.nan, m2m7=np.nan,
    m2m8=np.nan ,totalPL=0, cumReturns=0, balance=0
)

# only keep values between 9:15 to 15:30
df = df[(df['Datetime'].dt.time >= time(9, 15)) & (df['Datetime'].dt.time <= time(15, 30))]

# drop rows with same Datetime
df.drop_duplicates(subset='Datetime', keep='first', inplace=True)

# reset df index
df.reset_index(drop=True, inplace=True)


C:\Users\Aviral Mishra\AppData\Local\Temp\ipykernel_10344\1964448959.py:2: DtypeWarning: Columns (8,9,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'D:/all_data/data/{year}/NiftySpotData-{year}-1min-Signal-Expiry.csv')


In [4]:
# account for missing optionsData by dropping the rows from df where Datetime is
# dates_to_drop = ['2023-02-16', '2023-03-07', '2023-03-30', '2023-04-04', '2023-09-19', '2023-04-25', '2023-01-20', '2023-10-22', '2023-10-24']
# dates_to_drop = ['2022-09-22']
# dates_to_drop = ['2021-01-08', '2021-02-24', '2021-07-16', '2021-08-02', '2021-11-03', '2021-11-04']
# dates_to_drop = ['2020-01-31', '2020-03-13']
dates_to_drop = ['2024-03-02']
for date in dates_to_drop:
    df = df[df['Datetime'].dt.date != pd.to_datetime(date).date()]

df.reset_index(drop=True, inplace=True)

In [5]:
df.head()

,High,Low,Open,Close,Datetime,ama,rsi,atr,signal,closest_expiry,...,m2m2,m2m3,m2m4,m2m5,m2m6,m2m7,m2m8,totalPL,cumReturns,balance
0,21737.35,21684.75,21727.75,21700.80,2024-01-01 09:15:00+05:30,NaN,NaN,52.6,Hold,2024-01-04 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,21720.00,21695.35,21711.50,21695.65,2024-01-01 09:16:00+05:30,NaN,NaN,52.6,Hold,2024-01-04 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,21711.75,21694.75,21697.70,21709.55,2024-01-01 09:17:00+05:30,NaN,NaN,52.6,Hold,2024-01-04 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,21712.50,21698.40,21709.55,21699.55,2024-01-01 09:18:00+05:30,NaN,NaN,52.6,Hold,2024-01-04 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,21708.00,21693.55,21704.35,21693.75,2024-01-01 09:19:00+05:30,NaN,NaN,52.6,Hold,2024-01-04 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [6]:
df.tail()


,High,Low,Open,Close,Datetime,ama,rsi,atr,signal,closest_expiry,...,m2m2,m2m3,m2m4,m2m5,m2m6,m2m7,m2m8,totalPL,cumReturns,balance
34864,22473.15,22468.6,22472.15,22469.95,2024-05-17 15:25:00+05:30,22384.555014,59.023836,26.021429,Hold,2024-05-02 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
34865,22470.65,22464.4,22468.80,22469.95,2024-05-17 15:26:00+05:30,22384.555014,59.023836,26.021429,Hold,2024-05-02 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
34866,22482.40,22469.9,22469.90,22472.75,2024-05-17 15:27:00+05:30,22384.555014,59.023836,26.021429,Hold,2024-05-02 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
34867,22475.00,22461.2,22472.90,22465.30,2024-05-17 15:28:00+05:30,22384.555014,59.023836,26.021429,Hold,2024-05-02 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
34868,22468.30,22461.0,22466.75,22465.55,2024-05-17 15:29:00+05:30,22384.555014,59.023836,26.021429,Hold,2024-05-02 00:00:00+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [7]:
# optionsData = pd.read_csv(f'data/{year}/NiftyOptionsData-{month}{year}-1min.csv')
optionsData = pd.read_csv(f'D:/all_data/data/{year}/NiftyOptionsData-{year}-1min.csv')
optionsData['Datetime'] = pd.to_datetime(optionsData['Datetime'], format='%Y-%m-%d %H:%M:%S')
# optionsData['Datetime'] = optionsData['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
# optionsData['Datetime'] = pd.to_datetime(optionsData['Datetime'], format='%Y-%m-%d %H:%M:%S')
optionsData['ExpiryDate'] = pd.to_datetime(optionsData['ExpiryDate'], format='%Y-%m-%d %H:%M:%S%z')
optionsData['ExpiryDate'] = optionsData['ExpiryDate'].dt.strftime('%Y-%m-%d %H:%M:%S%z')
optionsData['ExpiryDate'] = pd.to_datetime(optionsData['ExpiryDate'], format='%Y-%m-%d %H:%M:%S%z')



optionsData = optionsData.sort_values(by='Datetime')
optionsData.reset_index(drop=True, inplace=True)

C:\Users\Aviral Mishra\AppData\Local\Temp\ipykernel_10344\367395002.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  optionsData = pd.read_csv(f'D:/all_data/data/{year}/NiftyOptionsData-{year}-1min.csv')


In [8]:
optionsData

,Datetime,Open,High,Low,Close,Ticker,ExpiryDate,Strike,Instrument Type,Symbol
0,2024-01-01 09:15:00,366.00,366.00,366.00,366.00,NIFTY24MAR21500PE.NFO,2024-03-28 00:00:00+00:00,21500.0,PE,NaN
1,2024-01-01 09:15:00,683.95,683.95,665.00,665.00,NIFTY24JAN21300CE.NFO,2024-01-25 00:00:00+00:00,21300.0,CE,NaN
2,2024-01-01 09:15:00,4.00,4.95,2.55,3.50,NIFTY2410420900PE.NFO,2024-01-04 00:00:00+00:00,20900.0,PE,NaN
3,2024-01-01 09:15:00,1.30,1.35,1.05,1.30,NIFTY2410418450PE.NFO,2024-01-04 00:00:00+00:00,18450.0,PE,NaN
4,2024-01-01 09:15:00,486.85,486.85,486.85,486.85,NIFTY24JAN21550CE.NFO,2024-01-25 00:00:00+00:00,21550.0,CE,NaN
...,...,...,...,...,...,...,...,...,...,...
18610835,2024-04-29 15:30:00,60.45,60.45,60.45,60.45,NIFTY30MAY2421700PE.NFO,2024-05-30 00:00:00+00:00,21700.0,PE,NIFTY
18610836,2024-04-29 15:30:00,167.35,167.35,167.35,167.35,NIFTY16MAY2422550PE.NFO,2024-05-16 00:00:00+00:00,22550.0,PE,NIFTY
18610837,2024-04-29 15:30:00,1.50,1.50,1.50,1.50,NIFTY09MAY2424200CE.NFO,2024-05-09 00:00:00+00:00,24200.0,CE,NIFTY
18610838,2024-04-29 15:30:00,15.85,15.85,15.85,15.85,NIFTY30MAY2420800PE.NFO,2024-05-30 00:00:00+00:00,20800.0,PE,NIFTY


In [9]:
optionsData.tail()

,Datetime,Open,High,Low,Close,Ticker,ExpiryDate,Strike,Instrument Type,Symbol
18610835,2024-04-29 15:30:00,60.45,60.45,60.45,60.45,NIFTY30MAY2421700PE.NFO,2024-05-30 00:00:00+00:00,21700.0,PE,NIFTY
18610836,2024-04-29 15:30:00,167.35,167.35,167.35,167.35,NIFTY16MAY2422550PE.NFO,2024-05-16 00:00:00+00:00,22550.0,PE,NIFTY
18610837,2024-04-29 15:30:00,1.50,1.50,1.50,1.50,NIFTY09MAY2424200CE.NFO,2024-05-09 00:00:00+00:00,24200.0,CE,NIFTY
18610838,2024-04-29 15:30:00,15.85,15.85,15.85,15.85,NIFTY30MAY2420800PE.NFO,2024-05-30 00:00:00+00:00,20800.0,PE,NIFTY
18610839,2024-04-29 15:30:00,257.75,257.75,257.75,257.75,NIFTY02MAY2422450CE.NFO,2024-05-02 00:00:00+00:00,22450.0,CE,NIFTY


In [21]:
# updated get close price fucntion
def get_close_price(strike_price, option_type, current_datetime, expiry_date):
    try:
        # First filter based on the first criteria
        
        # filtered_opd = optionsData.loc[(optionsData['Datetime'] == current_datetime) & (optionsData['Strike'] == strike_price) & (optionsData['ExpiryDate'] == expiry_date) & (optionsData['Instrument Type'] == option_type)]
        
        expiry_date = pd.to_datetime(expiry_date).date()
        filtered_opd = optionsData[optionsData['Datetime'] == current_datetime]
        filtered_opd = filtered_opd[filtered_opd['Instrument Type'] == option_type]
        filtered_opd = filtered_opd[filtered_opd['Strike'] == strike_price]
        filtered_opd['ExpiryDate'] = pd.to_datetime(filtered_opd['ExpiryDate']).dt.date
        filtered_opd = filtered_opd[filtered_opd['ExpiryDate'] == expiry_date]
        
        

        # Now, apply the second criteria on the filtered DataFrame
        result = filtered_opd['Close']

        if result.empty:
            logging.warning(f"No close price found for {strike_price}, {option_type} at {current_datetime} for expiry {expiry_date}")
            return 0
        else:
            return result.values[0]
    except Exception as e:
        logging.error(f"Error while fetching close price for {strike_price} at {current_datetime}")
        logging.error(e)



# get close price dict
def get_close_price_dict(atm_sp, wing_call, wing_put, sotmSPcall, sotmSPput, botmSPcall, botmSPput, current_datetime, expiry_date, i, df):
    try:
        # Define a list to store the futures
        futures = []

        # Create a ThreadPoolExecutor with a maximum of 4 threads
        with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
            # Submit tasks to fetch close prices asynchronously
            futures.append(executor.submit(get_close_price, atm_sp, 'CE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, atm_sp, 'PE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, wing_call, 'CE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, wing_put, 'PE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, sotmSPcall, 'CE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, sotmSPput, 'PE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, botmSPcall, 'CE', current_datetime, expiry_date))
            futures.append(executor.submit(get_close_price, botmSPput, 'PE', current_datetime, expiry_date))

        # Retrieve the results
        results = [future.result() for future in futures]
        
        try:
            is_data_valid = True
            for j in range(len(results)):
                if results[j] == False:
                    previous_value = df.at[i - 1, f'legPriceOrignal{j + 1}']
                    if np.isnan(previous_value):
                        if row.Datetime.time() == (9, 45):
                            is_data_valid = False
                            print(f"Data invalid at 9:45 for index {j}, skipping to next day.")
                        else:
                            results[j] = 0
                    else:
                        results[j] = previous_value
        except Exception as e:
            print(f"Error in get_close_price_dict: {e}")

        # Map the results to the ticker names
        close_prices = {
            'atmSPCall': results[0],
            'atmSPPut': results[1],
            'wingCallPrice': results[2],
            'wingPutPrice': results[3],
            'sotmCallPrice': results[4],
            'sotmPutPrice': results[5],
            'botmCallPrice': results[6],
            'botmPutPrice': results[7]
            
        }
        
        
        return close_prices, df
    except Exception as e:
        logging.error(f"Error in get_close_price_dict: {e}")

In [11]:
# positions
positions = {'beginx': [-1, -1, 1, 1, 0, 0, 0, 0], 'buy': [0, -2, 2, 2, -4, 0, 3, 0], 'sell': [-2, 0, 2, 2, 0, -4, 0, 3], 'squareoff': [0, 0, 0, 0, 0, 0, 0, 0], 'hard-squareoff': [0, 0, 0, 0, 0, 0, 0, 0]}

In [22]:
# m2m calculation
num_legs = 8

columns_to_reset = [ 'atmSP', 'wingCall', 'wingPut','sotmcall','sotmput','botmcall','botmput' , 'legPriceOrignal1', 'legPriceOrignal2',
    'legPriceOrignal3', 'legPriceOrignal4', 'legPriceOrignal5', 'legPriceOrignal6',
    'legPriceOrignal7', 'legPriceOrignal8', 'balance', 'lpos1', 'lpos2', 'lpos3',
    'lpos4','lpos5','lpos6','lpos7','lpos8', 'legAfterPos1', 'legAfterPos2', 'legAfterPos3', 'legAfterPos4','legAfterPos5', 'legAfterPos6', 'legAfterPos7', 'legAfterPos8',
    'legAfterPosDiff1', 'legAfterPosDiff2', 'legAfterPosDiff3', 'legAfterPosDiff4','legAfterPosDiff5', 'legAfterPosDiff6', 'legAfterPosDiff7', 'legAfterPosDiff8',
    'legPriceFinal1', 'legPriceFinal2', 'legPriceFinal3', 'legPriceFinal4','legPriceFinal5', 'legPriceFinal6', 'legPriceFinal7', 'legPriceFinal8',
    'm2m1', 'm2m2', 'm2m3', 'm2m4','m2m5', 'm2m6', 'm2m7', 'm2m8', 'totalPL', 'cumReturns'
]

lpos_column_names = [f"lpos{i+1}" for i in range(num_legs)]


def calculate_m2m_new(index, start_day=False, caller='o'):
    if str(df.at[index, 'position']).lower() == 'hold':
        df.at[index, 'totalPL'] = 0
        df.at[index, 'cumReturns'] = df.at[index-1, 'cumReturns']
        df.at[index, 'balance'] = df.at[index-1, 'balance']
        return
    position_to_take = positions[str(df.at[index, 'position'])]
    
    
    for i in range(0, num_legs):
        df.at[index, f"lpos{i+1}"] = position_to_take[i]
        # Calculating the leg after position
        df.at[index, f"legAfterPos{i+1}"] = df.at[index, f"legPriceOrignal{i+1}"] * df.at[index, f"lpos{i+1}"]
        

        # Calculating the difference between the leg after position and the previous leg after position
        difference_in_position = df.at[index, f"lpos{i+1}"] - df.at[index-1, f"lpos{i+1}"]
        df.at[index, f"legAfterPosDiff{i+1}"] = difference_in_position * df.at[index, f"legPriceOrignal{i+1}"]

        # Calculating Final Leg Price
        df.at[index, f"legPriceFinal{i+1}"] = df.at[index, f"legAfterPosDiff{i+1}"] + df.at[index-1, f"legAfterPos{i+1}"]

        if not start_day:
            # Calculating m2m
            df.at[index, f"m2m{i+1}"] = df.at[index, f"legPriceFinal{i+1}"] - df.at[index, f"legAfterPos{i+1}"]
        else:
            df.at[index, f"m2m{i+1}"] = 0

        
        # calculating the current data pnl which is the sum of all m2ms
        df.at[index, 'totalPL'] += df.at[index, f"m2m{i+1}"]

    # multiplying the totalPL by 50 for the lot size
    df.at[index, 'totalPL'] = df.at[index, 'totalPL'] * -1 * 50
    df.at[index, 'cumReturns'] = df.at[index-1, 'cumReturns'] + df.at[index, 'totalPL'] 
    # updating the balance
    df.at[index, 'balance'] = df.at[index-1, 'balance'] + df.at[index, 'totalPL']

In [23]:
squareoff_columns_reset = ['balance', 'lpos1', 'lpos2', 'lpos3','lpos4','lpos5','lpos6','lpos7','lpos8',
    'legAfterPos1', 'legAfterPos2', 'legAfterPos3', 'legAfterPos4',
    'legAfterPosDiff1', 'legAfterPosDiff2', 'legAfterPosDiff3', 'legAfterPosDiff4','legAfterPosDiff5', 'legAfterPosDiff6', 'legAfterPosDiff7', 'legAfterPosDiff8',
    'legPriceFinal1', 'legPriceFinal2', 'legPriceFinal3', 'legPriceFinal4','legPriceFinal5', 'legPriceFinal6', 'legPriceFinal7', 'legPriceFinal8',
    'm2m1', 'm2m2', 'm2m3', 'm2m4', 'm2m5', 'm2m6', 'm2m7', 'm2m8','totalPL', 'cumReturns']
    
# round to nearest 50
def round_to_nearest_50(number):
    return round(number / 50) * 50
    
for i in range(1, len(df)):
    df.at[df.index[i], 'atmSP'] = round_to_nearest_50(df.at[df.index[i], 'Close'])

In [24]:
# Function to determine if the current time is before the target time

def is_before_target_time(current_datetime, target_time="15:10:00+05:30"):
    target_time = datetime.strptime(target_time, "%H:%M:%S%z")
    # Compare the time parts
    return current_datetime.time() < target_time.time()


def get_daily_diff(current_datetime):
    day_of_week = current_datetime.weekday()
    # Define a dictionary to map each day to its corresponding difference
    day_diff_mapping = {
        "Monday": 300,
        "Tuesday": 250,
        "Wednesday": 200,
        "Thursday": 100,
        "Friday": 400,
        "Saturday": 0,
        "Sunday": 0
    }
    # Return the difference based on the day of the week
    return day_diff_mapping[datetime.strftime(current_datetime, '%A')]

In [25]:
# function to save data to csv
def save_data():
    # df.to_csv(f'data/backtest/backtest{month}{year}-adaptiveSL-1min.csv')
    df.to_csv(f'D:/all_data/data/backtest/backtest{year}-adaptiveSL-weeklybutteradj.csv')



In [26]:
logging.basicConfig(filename='logs.log', filemode='w', encoding='utf-8', level=logging.DEBUG, force=True)
position = "squareoff"

# set the initial capital and the current capital hi hirak
capital = 100000

logging.info("*"*100)
lotOrdercount = 0
first_run = True
highest_m2m = 0
week_started = False
is_data_valid = True

for i, row in enumerate(df.itertuples()):
    # timer_start = tm.time()
    # Check if this a new day and time is 09:30:00+05:30

    current_datetime_str = str(row.Datetime)[:-6]
    current_datetime_str_short = current_datetime_str
    expiry_date = row.closest_expiry
    #change expiry datetime is 15:15:00+05:30 of the row's closest expiry
    #expiry_datetime = row.closest_expiry.replace(hour=15, minute=15, second=0, microsecond=0)
    expiry_datetime = row.closest_expiry.replace(hour=9, minute=45, second=0, microsecond=0)
    expiry_datetime = expiry_datetime.astimezone(timezone(timedelta(hours=5, minutes=30)))
    has_traded_today = False
    has_squareoff_today = False
    
    logging.info(f"Current Datetime: {current_datetime_str}")

    # check if the time is 09:30:00+05:30 or it is the first run
    if '09:45:00' in current_datetime_str or first_run:
        logging.info(f"Position: {position} at {current_datetime_str}, 9:45 Entry")
        highest_m2m = 0
        # Calculate the ATM strike price
        atmSP = round_to_nearest_50(row.Close)
        daily_diff = get_daily_diff(row.Datetime)
        wingPut = atmSP - daily_diff
        wingCall = atmSP + daily_diff
        sotmcall = wingCall +  daily_diff
        sotmput = wingPut - daily_diff
        botmcall = sotmcall +  daily_diff
        botmput = sotmput - daily_diff
        
        
        # marking the postion as 0 updating tickers_dict
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, sotmcall, sotmput, botmcall, botmput, current_datetime_str_short, row.closest_expiry, i, df)

        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'sotmcall','sotmput', 'botmcall','botmput', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4','legPriceOrignal5', 'legPriceOrignal6', 'legPriceOrignal7', 'legPriceOrignal8']] = [atmSP, wingCall, wingPut, sotmcall,sotmput,botmcall,botmput, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice'],close_price_dict['sotmCallPrice'], close_price_dict['sotmPutPrice'], close_price_dict['botmCallPrice'], close_price_dict['botmPutPrice']]
        if first_run:
            logging.info(f"First run at {current_datetime_str}")
            # Reset values to 0 for the specified columns at row i
            df.loc[i, columns_to_reset] = 0
            df.at[i, 'position'] = position = 'hold'
            df.at[i, 'balance'] = capital
            first_run = False
        else:
            if not week_started:
                if not is_data_valid:
                    logging.info(f"Data invalid at {current_datetime_str}, unable to take 'beginx', skipping to next day.")
                    continue  # Skip the rest of the loop if data is invalid
                
                logging.info(f"Taking beginx at {current_datetime_str}, 9:45 Entry, week started part")
                df.at[i, 'position'] = 'beginx'
                week_started = True
                position = 'hold'
                df.at[i, 'cumReturns'] = 0
                calculate_m2m_new(i, start_day=True, caller='beginx 9:45')
                logging.info(f"Position: {position} at {current_datetime_str}, 9:45 Entry")
                df.at[i, 'stoploss'] = highest_m2m - (100000*0.05)
                lotOrdercount = lotOrdercount + 4
            else:
                df.at[i, 'position'] = position = df.at[i-1, 'position']
                calculate_m2m_new(i, start_day=True, caller='non beginx 9:45')

    elif df.at[i-1, 'position'] == 'hard-squareoff':
        # add 1 day to df.at[i-1, 'closest_expiry']
        if row.Datetime > df.at[i-1, 'closest_expiry'] + timedelta(days=1) and week_started == True:
            logging.info(f"Position: {position} at {current_datetime_str}, After HARD SQUAREOFF ******* ENTRY")
            df.at[i, 'position'] = position = df.at[i-1, 'position']
            df.at[i, 'balance'] = df.at[i-1, 'balance']
            df.at[i, 'cumReturns'] = 0
            df.at[i, 'totalPL'] = 0
            week_started = False
               
        else:
            df.at[i, 'position'] = position = df.at[i-1, 'position']
            df.at[i, 'balance'] = df.at[i-1, 'balance']
            df.at[i, 'cumReturns'] = df.at[i-1, 'cumReturns']
            df.at[i, 'totalPL'] = 0

    
    # if time is 15:15:00+05:30 then squareoff all positions
    elif row.Datetime == expiry_datetime :
        # buy atmSp call -> api call # buy atmSp put -> api call # sell wingPut -> api call # sell wingCall -> api call
        capital = df.at[i-1, 'balance']
        df.at[i, 'balance'] = df.at[i-1, 'balance']
        df.at[i, 'position'] = position = 'squareoff'
        if df.at[i-1, 'position'] == 'buy' or df.at[i-1, 'position'] == 'sell':
            lotOrdercount = lotOrdercount + 13
        elif df.at[i-1, 'position'] == 'beginx':
            lotOrdercount = lotOrdercount + 4 

        # store the values in the dataframe
        week_started = False
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, sotmcall, sotmput, botmcall, botmput, current_datetime_str_short, row.closest_expiry, i, df)
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'sotmcall','sotmput', 'botmcall','botmput', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4','legPriceOrignal5', 'legPriceOrignal6', 'legPriceOrignal7', 'legPriceOrignal8']] = [atmSP, wingCall, wingPut, sotmcall,sotmput,botmcall,botmput, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice'],close_price_dict['sotmCallPrice'], close_price_dict['sotmPutPrice'], close_price_dict['botmCallPrice'], close_price_dict['botmPutPrice']]
        calculate_m2m_new(i, caller='15:15')
        df.at[i, 'cumReturns'] = 0

    elif row.Datetime.date() == expiry_date and row.Datetime.time() <= time(15, 30) and row.Datetime.time() > time(15, 15) :
        # store the values in the dataframe
        df.loc[i, ['atmSP', 'wingCall', 'wingPut','sotmcall','sotmput', 'botmcall','botmput'] ] = [atmSP, wingCall, wingPut,sotmcall, sotmput, botmcall, botmput]
        df.at[df.index[i], 'position'] = position =  'squareoff'
        df.at[i, 'balance'] = df.at[i-1, 'balance'] 
    
    elif df.at[i-1, 'position'] == 'squareoff' :

        # check if current Datetime minutes is a 00, 15, 30, 45
        if row.Datetime.minute % 15 == 0 and row.Datetime.time() > time(9, 45) and row.Datetime.time() < time(15, 15):
            # Calculate the ATM strike price
            atmSP = round_to_nearest_50(row.Close)
            daily_diff = get_daily_diff(row.Datetime)
            wingPut = atmSP - daily_diff
            wingCall = atmSP + daily_diff
            sotmcall = wingCall +  daily_diff
            sotmput = wingPut - daily_diff
            botmcall = sotmcall +  daily_diff
            botmput = sotmput - daily_diff
            close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, sotmcall, sotmput, botmcall, botmput, current_datetime_str_short, row.closest_expiry, i, df)
            df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'sotmcall','sotmput', 'botmcall','botmput', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4','legPriceOrignal5', 'legPriceOrignal6', 'legPriceOrignal7', 'legPriceOrignal8']] = [atmSP, wingCall, wingPut, sotmcall,sotmput,botmcall,botmput, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice'],close_price_dict['sotmCallPrice'], close_price_dict['sotmPutPrice'], close_price_dict['botmCallPrice'], close_price_dict['botmPutPrice']]
            df.at[i, 'position'] = position = 'beginx'
            calculate_m2m_new(i, start_day=True, caller='beginx')
            df.at[i,'cumReturns'] = 0
            df.at[i, 'balance'] = df.at[i-1, 'balance']
            df.at[i, 'totalPL'] = 0
            df.at[i, 'stoploss'] = highest_m2m - (100000*0.05)
            logging.info(f"Position: {position} at {current_datetime_str}, beginx after rsi exit")
            lotOrdercount = lotOrdercount + 4
            week_started = True
        else:
            df.at[i, 'position'] = position = 'squareoff'
            df.at[i, 'balance'] = df.at[i-1, 'balance']
            df.at[i, 'cumReturns'] = df.at[i-1, 'cumReturns']
            df.at[i, 'totalPL'] = 0
            week_started = False
    
    else:
        close_price_dict, df = get_close_price_dict(atmSP, wingCall, wingPut, sotmcall, sotmput, botmcall, botmput, current_datetime_str_short, row.closest_expiry, i, df)
        if position == 'hold':
            if row.signal == 'Buy':
                if row.rsi < 70 and row.ama < row.Close:
                    # buy atmSp call -> api call # Sell atmSp put -> api call # buy wingCall -> api call # buy wingPut -> api call
                    df.at[i, 'position'] = position = 'buy'
                    has_traded_today = True
                    lotOrdercount = lotOrdercount + 14
            
            elif row.signal == 'Sell':
                if row.rsi > 30 and row.ama > row.Close:
                    # sell atmSp call -> api call # buy atmSp put -> api call # Buy wingPut -> api call # Buy wingCall -> api call
                    df.at[i, 'position'] = position = 'sell'
                    has_traded_today = True
                    lotOrdercount = lotOrdercount + 14

        elif position == 'buy':
            if row.signal == 'Hold':
                if row.rsi > 70:
                    # buy atmSp put -> api call x 2 # sell wingPut -> api call x 2 # sell wingCall -> api call x 2
                    df.at[i, 'position'] = position = 'squareoff'
                    has_traded_today = True
            
            elif row.signal == 'Sell':          
                # buy atmSp put -> api call # sell wingPut -> api call # sell wingCall -> api call
                df.at[i, 'position'] = position =  'squareoff'
                has_traded_today = True

        elif position == 'sell':
            if row.signal == 'Hold':
                if row.rsi < 30:              
                    # buy atmSp call -> api call # sell wingPut -> api call # sell wingCall -> api call  
                    df.at[i, 'position'] = position = 'squareoff'
                    has_traded_today = True

                    
            elif row.signal == 'Buy':
                df.at[df.index[i], 'position'] = position = 'squareoff'
                has_traded_today = True
                

        df.at[i, 'position'] = position
        df.loc[i, ['atmSP', 'wingCall', 'wingPut', 'sotmcall','sotmput', 'botmcall','botmput', 'legPriceOrignal1', 'legPriceOrignal2', 'legPriceOrignal3', 'legPriceOrignal4','legPriceOrignal5', 'legPriceOrignal6', 'legPriceOrignal7', 'legPriceOrignal8']] = [atmSP, wingCall, wingPut, sotmcall, sotmput, botmcall, botmput, close_price_dict['atmSPCall'], close_price_dict['atmSPPut'], close_price_dict['wingCallPrice'], close_price_dict['wingPutPrice'],close_price_dict['sotmCallPrice'], close_price_dict['sotmPutPrice'], close_price_dict['botmCallPrice'], close_price_dict['botmPutPrice']]
        
            

        if not has_traded_today:
            df.at[i, 'position'] = df.at[i-1, 'position']

        
        # called everytime
        calculate_m2m_new(i, caller='daily')

        if df.at[i, 'cumReturns'] > highest_m2m:
            highest_m2m = df.at[i, 'cumReturns']
        
        df.at[i, 'stoploss'] = stoploss_price = highest_m2m - (100000*0.05)

        if df.at[i, 'cumReturns'] < stoploss_price:
            if df.at[i, 'cumReturns'] < 0:
                logging.info(f" Stoploss hit at {current_datetime_str} with highest m2m: {highest_m2m} and stoploss price: {stoploss_price}")
                logging.info(f" Stoploss hit at {current_datetime_str}")
                # squareoff all positions
                df.loc[i, squareoff_columns_reset] = 0
                df.at[i, 'position'] = position = 'hard-squareoff'
                calculate_m2m_new(i, caller='hard-squareoff')
                highest_m2m = df.at[i, 'cumReturns']
                if df.at[i-1, 'position'] == 'buy' or df.at[i-1, 'position'] == 'sell':
                    lotOrdercount = lotOrdercount + 13
                elif df.at[i-1, 'position'] == 'beginx':
                    lotOrdercount = lotOrdercount + 4    
            elif df.at[i, 'cumReturns'] > 0:

                
                logging.info(f" Stoploss hit at {current_datetime_str} with highest m2m: {highest_m2m} and stoploss price: {stoploss_price}")
                logging.info(f" Stoploss hit at {current_datetime_str}")
                # squareoff all positions
                df.loc[i, squareoff_columns_reset] = 0
                df.at[i, 'position'] = position = 'squareoff'
                highest_m2m = df.at[i, 'cumReturns']
                df.at[i, 'totalPL'] = 0
                df.at[i, 'stoploss'] = stoploss_price = highest_m2m - (100000*0.05)
                df.at[i, 'cumReturns'] = 0
                df.at[i, 'balance'] = df.at[i-1, 'balance']
                if df.at[i-1, 'position'] == 'buy' or df.at[i-1, 'position'] == 'sell':
                    lotOrdercount = lotOrdercount + 13
                elif df.at[i-1, 'position'] == 'beginx':
                    lotOrdercount = lotOrdercount + 4 
            
            
            
    
    # timer_end = tm.time()
    # logging.info(f"Time taken for this iteration: {timer_end - timer_start} seconds")


    # df.at[df.index[i], 'position'] = position


print(f"Lot Order Count: {lotOrdercount}")
save_data()





C:\Users\Aviral Mishra\AppData\Local\Temp\ipykernel_10344\2948123802.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.549999999999983' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'totalPL'] += df.at[index, f"m2m{i+1}"]
C:\Users\Aviral Mishra\AppData\Local\Temp\ipykernel_10344\2948123802.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-34.999999999999254' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'cumReturns'] = df.at[index-1, 'cumReturns'] + df.at[index, 'totalPL']
C:\Users\Aviral Mishra\AppData\Local\Temp\ipykernel_10344\2948123802.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '99967.5' has dtype incompatibl

KeyboardInterrupt: 

---


In [149]:
save_data()
